In [4]:
import ipywidgets as widgets
from IPython.display import display
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
from skimage import io as skio, filters # Alias skimage.io to skio
from skimage.feature import peak_local_max
from scipy.spatial.distance import euclidean
from skimage.exposure import equalize_hist
import io # Explicitly import the standard Python io module


class FingerprintMatcher:
    def __init__(self):
        # Create output widget for all messages
        self.output_widget = widgets.Output()
        display(self.output_widget)

        # Initialize state
        self.ref_image = None
        self.samp_image = None

        # Create upload widgets
        self.ref_upload = widgets.FileUpload(
            description='Reference Image',
            accept='.jpg,.jpeg,.png,.bmp',
            multiple=False
        )
        self.samp_upload = widgets.FileUpload(
            description='Sample Image',
            accept='.jpg,.jpeg,.png,.bmp',
            multiple=False
        )

        # Create process and match buttons
        self.process_btn = widgets.Button(description='Process Images')
        self.match_btn = widgets.Button(description='Match Fingerprints')

        # Connect button clicks to functions
        self.process_btn.on_click(self._process_images)
        self.match_btn.on_click(self._match_fingerprints)

        # Display all widgets
        with self.output_widget:
            print("Fingerprint Matcher initialized")
            display(self.ref_upload)
            display(self.samp_upload)
            display(self.process_btn)
            display(self.match_btn)

    def _preprocess_image(self, image_path):
        with self.output_widget:
            print(f"Processing image")
            try:
                # Read image using PIL from bytes
                img_bytes = image_path.tobytes()
                img = Image.open(io.BytesIO(img_bytes)) # Use standard Python io.BytesIO

                # Convert to grayscale if needed
                if img.mode != 'L':
                    img = img.convert('L')

                # Convert to numpy array
                img_array = np.array(img)

                # Apply histogram equalization
                enhanced_img = equalize_hist(img_array)

                return enhanced_img
            except Exception as e:
                print(f"Error processing image: {str(e)}")
                return None


    def _extract_features(self, processed_image):
        with self.output_widget:
            print("Extracting features...")
            try:
                # Ridge detection using Sobel filters
                sobel_x = filters.sobel_h(processed_image)
                sobel_y = filters.sobel_v(processed_image)

                # Combine gradients
                gradient = np.sqrt(sobel_x**2 + sobel_y**2)

                # Find local maxima for minutiae points
                min_distance = 10
                # threshold = 0.4 * gradient.max()  <- REMOVED
                coordinates = peak_local_max(gradient, min_distance=min_distance,
                                                    # threshold=threshold) <- REMOVED
                                                    )

                return coordinates
            except Exception as e:
                print(f"Error extracting features: {str(e)}")
                return None

    def _match_features(self, ref_points, samp_points):
        with self.output_widget:
            print("Matching features...")
            try:
                # Calculate distances between all point pairs
                distances = []
                for ref_point in ref_points:
                    for samp_point in samp_points:
                        dist = euclidean(ref_point, samp_point)
                        distances.append(dist)

                # Threshold for acceptable matches
                threshold = 10
                matches = sum(d < threshold for d in distances)

                # Calculate similarity score
                max_possible_matches = min(len(ref_points), len(samp_points))
                similarity_score = (matches / max_possible_matches) * 100

                return similarity_score
            except Exception as e:
                print(f"Error during matching: {str(e)}")
                return None

    def _process_images(self, b):
        with self.output_widget:
            print("\n=== Processing Images ===")
            # Get uploaded files
            ref_files = self.ref_upload.value
            samp_files = self.samp_upload.value

            print(f"Type of ref_files: {type(ref_files)}")  # Debug print - Keep for now for verification
            print(f"Value of ref_files: {ref_files}")    # Debug print - Keep for now for verification
            print(f"Type of samp_files: {type(samp_files)}") # Debug print - Keep for now for verification
            print(f"Value of samp_files: {samp_files}")   # Debug print - Keep for now for verification


            if not ref_files or not samp_files:
                print("Error: Please upload both reference and sample images")
                return

            # Process reference image
            if ref_files:
                # Assume ref_files is a tuple of Bunch objects
                for ref_file_info in ref_files: # Iterate directly over the tuple
                    print(f"Processing reference file: {ref_file_info}") # Debug print - print the Bunch object
                    self.ref_image = self._preprocess_image(ref_file_info['content']) # Access 'content'
                    break  # Since multiple=False, process only the first file in the tuple

            # Process sample image
            if samp_files:
                # Assume samp_files is a tuple of Bunch objects
                for samp_file_info in samp_files: # Iterate directly over the tuple
                    print(f"Processing sample file: {samp_file_info}") # Debug print - print the Bunch object
                    self.samp_image = self._preprocess_image(samp_file_info['content']) # Access 'content'
                    break  # Since multiple=False, process only the first file in the tuple


            if self.ref_image is None or self.samp_image is None:
                print("Error: Failed to process one or both images")
                return

            # Display processed images
            fig, axes = plt.subplots(1, 2, figsize=(12, 5))
            axes[0].imshow(self.ref_image, cmap='gray')
            axes[0].set_title('Reference Image')
            axes[1].imshow(self.samp_image, cmap='gray')
            plt.show()
            print("Images processed successfully!")


    def _match_fingerprints(self, b):
        with self.output_widget:
            print("\n=== Matching Fingerprints ===")
            if self.ref_image is None or self.samp_image is None:
                print("Error: Please process images first")
                return

            # Extract features
            ref_features = self._extract_features(self.ref_image)
            samp_features = self._extract_features(self.samp_image)

            if ref_features is None or samp_features is None:
                print("Error: Failed to extract features")
                return

            # Match features
            score = self._match_features(ref_features, samp_features)

            if score is not None:
                print(f"\nMatching Results:")
                print(f"Similarity Score: {score:.2f}%")
            else:
                print("Error: Failed to calculate similarity score")

if __name__ == '__main__':
    fingerprint_matcher = FingerprintMatcher()

Output()

In [5]:
import ipywidgets as widgets
from IPython.display import display
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
from skimage import io as skio, filters # Alias skimage.io to skio
from skimage.feature import peak_local_max
from scipy.spatial.distance import euclidean
from skimage.exposure import equalize_hist
import io # Explicitly import the standard Python io module


class FingerprintMatcher:
    def __init__(self):
        # Create output widget for all messages
        self.output_widget = widgets.Output()
        display(self.output_widget)

        # Initialize state
        self.ref_image = None
        self.samp_image = None

        # Create upload widgets
        self.ref_upload = widgets.FileUpload(
            description='Reference Image',
            accept='.jpg,.jpeg,.png,.bmp',
            multiple=False
        )
        self.samp_upload = widgets.FileUpload(
            description='Sample Image',
            accept='.jpg,.jpeg,.png,.bmp',
            multiple=False
        )

        # Create process and match buttons
        self.process_btn = widgets.Button(description='Process Images')
        self.match_btn = widgets.Button(description='Match Fingerprints')

        # Connect button clicks to functions
        self.process_btn.on_click(self._process_images)
        self.match_btn.on_click(self._match_fingerprints)

        # Display all widgets
        with self.output_widget:
            print("Fingerprint Matcher initialized")
            display(self.ref_upload)
            display(self.samp_upload)
            display(self.process_btn)
            display(self.match_btn)

    def _preprocess_image(self, image_path):
        with self.output_widget:
            print(f"Processing image")
            try:
                # Read image using PIL from bytes
                img_bytes = image_path.tobytes()
                img = Image.open(io.BytesIO(img_bytes)) # Use standard Python io.BytesIO

                # Convert to grayscale if needed
                if img.mode != 'L':
                    img = img.convert('L')

                # Convert to numpy array
                img_array = np.array(img)

                # Apply histogram equalization
                enhanced_img = equalize_hist(img_array)

                return enhanced_img
            except Exception as e:
                print(f"Error processing image: {str(e)}")
                return None


    def _extract_features(self, processed_image):
        with self.output_widget:
            print("Extracting features...")
            try:
                # Ridge detection using Sobel filters
                sobel_x = filters.sobel_h(processed_image)
                sobel_y = filters.sobel_v(processed_image)

                # Combine gradients
                gradient = np.sqrt(sobel_x**2 + sobel_y**2)

                # Find local maxima for minutiae points
                min_distance = 10
                coordinates = peak_local_max(gradient, min_distance=min_distance)

                return coordinates
            except Exception as e:
                print(f"Error extracting features: {str(e)}")
                return None

    def _match_features(self, ref_points, samp_points):
        with self.output_widget:
            print("Matching features...")
            try:
                print(f"Type of ref_points: {type(ref_points)}, Shape: {ref_points.shape if isinstance(ref_points, np.ndarray) else 'N/A'}")
                print(f"Type of samp_points: {type(samp_points)}, Shape: {samp_points.shape if isinstance(samp_points, np.ndarray) else 'N/A'}")

                # Calculate distances between all point pairs
                distances = []
                for ref_point in ref_points:
                    for samp_point in samp_points:
                        dist = euclidean(ref_point, samp_point)
                        distances.append(dist)

                print(f"Type of distances: {type(distances)}") # Debug print for distances type
                if distances: # Check if distances is not empty before printing
                    print(f"Type of first distance in distances: {type(distances[0]) if distances else 'N/A'}") # Debug print for type of element in distances
                    if isinstance(distances[0], np.ndarray): # If it's an array, print its shape
                        print(f"Shape of first distance in distances: {distances[0].shape}")


                # Threshold for acceptable matches
                threshold = 10
                matches = sum(d < threshold for d in distances) # Potential error line

                # Calculate similarity score
                max_possible_matches = min(len(ref_points), len(samp_points))
                similarity_score = (matches / max_possible_matches) * 100

                return similarity_score
            except Exception as e:
                print(f"Error during matching: {str(e)}")
                return None

    def _process_images(self, b):
        with self.output_widget:
            print("\n=== Processing Images ===")
            # Get uploaded files
            ref_files = self.ref_upload.value
            samp_files = self.samp_upload.value

            print(f"Type of ref_files: {type(ref_files)}")  # Debug print - Keep for now for verification
            print(f"Value of ref_files: {ref_files}")    # Debug print - Keep for now for verification
            print(f"Type of samp_files: {type(samp_files)}") # Debug print - Keep for now for verification
            print(f"Value of samp_files: {samp_files}")   # Debug print - Keep for now for verification


            if not ref_files or not samp_files:
                print("Error: Please upload both reference and sample images")
                return

            # Process reference image
            if ref_files:
                # Assume ref_files is a tuple of Bunch objects
                for ref_file_info in ref_files: # Iterate directly over the tuple
                    print(f"Processing reference file: {ref_file_info}") # Debug print - print the Bunch object
                    self.ref_image = self._preprocess_image(ref_file_info['content']) # Access 'content'
                    break  # Since multiple=False, process only the first file in the tuple

            # Process sample image
            if samp_files:
                # Assume samp_files is a tuple of Bunch objects
                for samp_file_info in samp_files: # Iterate directly over the tuple
                    print(f"Processing sample file: {samp_file_info}") # Debug print - print the Bunch object
                    self.samp_image = self._preprocess_image(samp_file_info['content']) # Access 'content'
                    break  # Since multiple=False, process only the first file in the tuple


            if self.ref_image is None or self.samp_image is None:
                print("Error: Failed to process one or both images")
                return

            # Display processed images
            fig, axes = plt.subplots(1, 2, figsize=(12, 5))
            axes[0].imshow(self.ref_image, cmap='gray')
            axes[0].set_title('Reference Image')
            axes[1].imshow(self.samp_image, cmap='gray')
            plt.show()
            print("Images processed successfully!")


    def _match_fingerprints(self, b):
        with self.output_widget:
            print("\n=== Matching Fingerprints ===")
            if self.ref_image is None or self.samp_image is None:
                print("Error: Please process images first")
                return

            # Extract features
            ref_features = self._extract_features(self.ref_image)
            samp_features = self._extract_features(self.samp_image)

            if ref_features is None or samp_features is None:
                print("Error: Failed to extract features")
                return

            # Match features
            score = self._match_features(ref_features, samp_features)

            if score is not None:
                print(f"\nMatching Results:")
                print(f"Similarity Score: {score:.2f}%")
            else:
                print("Error: Failed to calculate similarity score")

if __name__ == '__main__':
    fingerprint_matcher = FingerprintMatcher()

Output()

In [6]:
import ipywidgets as widgets
from IPython.display import display
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
from skimage import io as skio, filters # Alias skimage.io to skio
from skimage.feature import peak_local_max
from scipy.spatial.distance import euclidean
from skimage.exposure import equalize_hist
import io # Explicitly import the standard Python io module


class FingerprintMatcher:
    def __init__(self):
        # Create output widget for all messages
        self.output_widget = widgets.Output()
        display(self.output_widget)

        # Initialize state
        self.ref_image = None
        self.samp_image = None
        self.ref_features = None  # To store features (coordinates, gradient)
        self.samp_features = None  # To store features (coordinates, gradient)


        # Create upload widgets
        self.ref_upload = widgets.FileUpload(
            description='Reference Image',
            accept='.jpg,.jpeg,.png,.bmp',
            multiple=False
        )
        self.samp_upload = widgets.FileUpload(
            description='Sample Image',
            accept='.jpg,.jpeg,.png,.bmp',
            multiple=False
        )

        # Create process and match buttons
        self.process_btn = widgets.Button(description='Process Images')
        self.match_btn = widgets.Button(description='Match Fingerprints')

        # Connect button clicks to functions
        self.process_btn.on_click(self._process_images)
        self.match_btn.on_click(self._match_fingerprints)

        # Display all widgets
        with self.output_widget:
            print("Fingerprint Matcher initialized")
            display(self.ref_upload)
            display(self.samp_upload)
            display(self.process_btn)
            display(self.match_btn)

    def _preprocess_image(self, image_path):
        with self.output_widget:
            print(f"Processing image")
            try:
                # Read image using PIL from bytes
                img_bytes = image_path.tobytes()
                img = Image.open(io.BytesIO(img_bytes)) # Use standard Python io.BytesIO

                # Convert to grayscale if needed
                if img.mode != 'L':
                    img = img.convert('L')

                # Convert to numpy array
                img_array = np.array(img)

                # Apply histogram equalization
                enhanced_img = equalize_hist(img_array)

                return enhanced_img
            except Exception as e:
                print(f"Error processing image: {str(e)}")
                return None


    def _extract_features(self, processed_image):
        with self.output_widget:
            print("Extracting features...")
            try:
                # Ridge detection using Sobel filters
                sobel_x = filters.sobel_h(processed_image)
                sobel_y = filters.sobel_v(processed_image)

                # Combine gradients (this is our ridge detail image)
                gradient = np.sqrt(sobel_x**2 + sobel_y**2)

                # Find local maxima for minutiae points
                min_distance = 10
                coordinates = peak_local_max(gradient, min_distance=min_distance)

                print(f"Number of minutiae detected: {len(coordinates)}") # Print num of minutiae

                return coordinates, gradient # Return both coordinates and gradient
            except Exception as e:
                print(f"Error extracting features: {str(e)}")
                return None, None # Return None for both in case of error

    def _match_features(self, ref_points, samp_points):
        with self.output_widget:
            print("Matching features...")
            try:
                print(f"Type of ref_points: {type(ref_points)}, Shape: {ref_points.shape if isinstance(ref_points, np.ndarray) else 'N/A'}")
                print(f"Type of samp_points: {type(samp_points)}, Shape: {samp_points.shape if isinstance(samp_points, np.ndarray) else 'N/A'}")

                # Calculate distances between all point pairs
                distances = []
                for ref_point in ref_points:
                    for samp_point in samp_points:
                        dist = euclidean(ref_point, samp_point)
                        distances.append(dist)

                print(f"Type of distances: {type(distances)}") # Debug print for distances type
                if distances: # Check if distances is not empty before printing
                    print(f"Type of first distance in distances: {type(distances[0]) if distances else 'N/A'}") # Debug print for type of element in distances
                    if isinstance(distances[0], np.ndarray): # If it's an array, print its shape
                        print(f"Shape of first distance in distances: {distances[0].shape}")


                # Threshold for acceptable matches
                threshold = 10
                matches = sum(d < threshold for d in distances) # Potential error line

                # Calculate similarity score
                max_possible_matches = min(len(ref_points), len(samp_points))
                similarity_score = (matches / max_possible_matches) * 100

                return similarity_score
            except Exception as e:
                print(f"Error during matching: {str(e)}")
                return None

    def _process_images(self, b):
        with self.output_widget:
            print("\n=== Processing Images ===")
            # Get uploaded files
            ref_files = self.ref_upload.value
            samp_files = self.samp_upload.value

            print(f"Type of ref_files: {type(ref_files)}")  # Debug print - Keep for now for verification
            print(f"Value of ref_files: {ref_files}")    # Debug print - Keep for now for verification
            print(f"Type of samp_files: {type(samp_files)}") # Debug print - Keep for now for verification
            print(f"Value of samp_files: {samp_files}")   # Debug print - Keep for now for verification


            if not ref_files or not samp_files:
                print("Error: Please upload both reference and sample images")
                return

            # Process reference image
            if ref_files:
                # Assume ref_files is a tuple of Bunch objects
                for ref_file_info in ref_files: # Iterate directly over the tuple
                    print(f"Processing reference file: {ref_file_info}") # Debug print - print the Bunch object
                    self.ref_image = self._preprocess_image(ref_file_info['content']) # Access 'content'
                    self.ref_features = self._extract_features(self.ref_image) # Extract features and store
                    break  # Since multiple=False, process only the first file in the tuple

            # Process sample image
            if samp_files:
                # Assume samp_files is a tuple of Bunch objects
                for samp_file_info in samp_files: # Iterate directly over the tuple
                    print(f"Processing sample file: {samp_file_info}") # Debug print - print the Bunch object
                    self.samp_image = self._preprocess_image(samp_file_info['content']) # Access 'content'
                    self.samp_features = self._extract_features(self.samp_image) # Extract features and store
                    break  # Since multiple=False, process only the first file in the tuple


            if self.ref_image is None or self.samp_image is None or self.ref_features is None or self.samp_features is None:
                print("Error: Failed to process one or both images or extract features")
                return

            # Display processed images and ridge details
            fig, axes = plt.subplots(2, 2, figsize=(12, 10)) # 2 rows, 2 columns now

            axes[0, 0].imshow(self.ref_image, cmap='gray') # Processed ref image
            axes[0, 0].set_title('Processed Reference Image')

            axes[0, 1].imshow(self.ref_features[1], cmap='gray') # Ridge detail ref image (gradient)
            axes[0, 1].set_title('Reference Ridge Detail')

            axes[1, 0].imshow(self.samp_image, cmap='gray') # Processed sample image
            axes[1, 0].set_title('Processed Sample Image')

            axes[1, 1].imshow(self.samp_features[1], cmap='gray') # Ridge detail sample image (gradient)
            axes[1, 1].set_title('Sample Ridge Detail')


            plt.tight_layout() # Adjust layout to prevent overlapping titles
            plt.show()
            print("Images and ridge details processed successfully!")


    def _match_fingerprints(self, b):
        with self.output_widget:
            print("\n=== Matching Fingerprints ===")
            if self.ref_image is None or self.samp_image is None or self.ref_features is None or self.samp_features is None:
                print("Error: Please process images first and ensure features are extracted")
                return

            # Extract features (already done in _process_images, using stored features)
            ref_coordinates = self.ref_features[0] # Get coordinates from stored features
            samp_coordinates = self.samp_features[0] # Get coordinates from stored features


            # Match features
            score = self._match_features(ref_coordinates, samp_coordinates)

            if score is not None:
                print(f"\nMatching Results:")
                print(f"Similarity Score: {score:.2f}%")

                # --- Visualization of Minutiae ---
                fig, axes = plt.subplots(1, 2, figsize=(12, 6)) # 1 row, 2 cols for minutiae viz

                axes[0].imshow(self.ref_image, cmap='gray')
                axes[0].set_title('Reference Image with Minutiae')
                axes[0].plot(ref_coordinates[:, 1], ref_coordinates[:, 0], 'ro', markersize=3) # Plot red dots, careful with x, y coordinate order


                axes[1].imshow(self.samp_image, cmap='gray')
                axes[1].set_title('Sample Image with Minutiae')
                axes[1].plot(samp_coordinates[:, 1], samp_coordinates[:, 0], 'ro', markersize=3) # Plot red dots, careful with x, y coordinate order


                plt.tight_layout()
                plt.show()
                print("Minutiae visualized!")


            else:
                print("Error: Failed to calculate similarity score")

if __name__ == '__main__':
    fingerprint_matcher = FingerprintMatcher()

Output()